## OLLAMA RAG
- Using OLLAMA & LLaMA to Analyze a Dataset or Document

#### 1. Data Ingestion
We begin by loading and chunking documents. For this demonstration, we use PyPDFLoader to load a PDF and split it into smaller, overlapping chunks for better retrieval accuracy.

In [1]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter

# Load the document
loader = PyPDFLoader("BillOfRights-US.pdf")
documents = loader.load()

# Split the document into chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=30, separator="\n")
docs = text_splitter.split_documents(documents=documents)

#### Now let's load data from Codebasics FAQ csv file

### 2. Data Embedding and Storage with FAISS
FAISS (Facebook AI Similarity Search) is a versatile and efficient library for vector similarity search. It enables scalable and fast retrieval of embeddings.

Choosing the Embedding Model

We use sentence-transformers/all-mpnet-base-v2, known for its robust performance across various text tasks. Alternatives like BGE or MiniLM can be used for specific trade-offs in speed vs. accuracy.

!pip install sentence-transformers

In [2]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

# Load embedding model
embedding_model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {"device": "cuda"}
embeddings = HuggingFaceEmbeddings(
    model_name=embedding_model_name,
    model_kwargs=model_kwargs
)

# Create FAISS vector store
vectorstore = FAISS.from_documents(docs, embeddings)

# Save and reload the vector store
vectorstore.save_local("faiss_index_")
persisted_vectorstore = FAISS.load_local("faiss_index_", embeddings, allow_dangerous_deserialization=True)

# Create a retriever
retriever = persisted_vectorstore.as_retriever()

/tmp/ipykernel_503/305371053.py:7: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
/home/aks/miniconda3/envs/llms/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 3. Loading LLaMA with Ollama
LLaMA (Large Language Model Meta AI) is a state-of-the-art model designed for versatile NLP tasks. Ollama simplifies deploying LLaMA models, even in constrained environments like Google Colab.

Installing Ollama

Run the following commands to install and set up Ollama:

In [ ]:
# Initializing LLaMA
# Using the LangChain Ollama integration, we load the LLaMA model:
from langchain_community.llms import Ollama

# Initialize the LLaMA model
llm = Ollama(model="llama3.2")

# Test with a sample prompt
response = llm.invoke("Tell me a joke")
print(response)

### 4. Orchestrating RAG with RetrievalQA
We combine the retriever and LLaMA into a RetrievalQA chain. This allows for interactive question-answering grounded in the ingested documents.

In [4]:
from langchain.chains import RetrievalQA

# Create RetrievalQA
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)


In [5]:
qa('what is first amandament says')

/tmp/ipykernel_503/1292279828.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  qa('what is first amandament says')


{'query': 'what is first amandament says',
 'result': 'The First Amendment says:\n\n"Congress shall make no law respecting an establishment of religion, or prohibiting the free exercise thereof; or abridging the freedom of speech, or of the press; or the right of the people peaceably to assemble, and to petition the Government for a redress of grievances."\n\nIn simpler terms, the First Amendment protects the following rights:\n\n1. Freedom of religion\n2. Freedom of speech\n3. Freedom of the press\n4. Right to peaceful assembly\n5. Right to petition the government for redress of grievances.'}

In [6]:
qa('what is first amandament means')

{'query': 'what is first amandament means',
 'result': 'The First Amendment to the United States Constitution states:\n\n"Congress shall make no law respecting an establishment of religion, or prohibiting the free exercise thereof; or abridging the freedom of speech, or of the press; or the right of the people peaceably to assemble, and to petition the Government for a redress of grievances."\n\nIn simpler terms, the First Amendment means that the government cannot:\n\n1. **Establish a national religion**: The government cannot create a law that favors one religion over another or forces people to practice a particular religion.\n2. **Restrict the free exercise of religion**: People have the right to practice their own religion freely, as long as it doesn\'t harm others.\n3. **Limit freedom of speech**: People have the right to express their opinions, ideas, and thoughts without fear of government censorship or punishment.\n4. **Restrict freedom of the press**: The government cannot co

### FIN